# Price Data Download Notebook

In [3]:
import os
from datetime import datetime

import yfinance as yf
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Literal
from tqdm import tqdm
from alpha_vantage.timeseries import TimeSeries

import src.utils.file_management as filemgmt
import src.pipeline.preprocessing as preprocessing

In [4]:
ROOT = Path().resolve().parent
DATA = ROOT / "data"
DAILY_PRICES = DATA / "daily_price_downloads"
MINUTELY_PRICES = DATA / "minutely_price_downloads"
INTERPOLATED_PRICES = DATA / "interpolated_prices"

AV_API_KEY_FILE = ROOT / "private" / "Alpha Vantage API Key.txt"
with open(AV_API_KEY_FILE) as file: AV_API_KEY = file.read()


## Download from AlphaVantage
(Restricted to 25 daily free requests)

### Execution

In [12]:
frame = preprocessing.get_data_from_alphavantage(AV_API_KEY,
                                   save_path=MINUTELY_PRICES,
                                   start_year_month=(2014, 7),
                                   end_year_month=(2016, 6),
                                   csv_path_to_concat=MINUTELY_PRICES)
frame

Will query the AlphaVantage API 24 times based on the specified time range. Queries:
 ['2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06']
Since provided csv_path_to_concat is a directory will now load: /Users/paulrusing/Library/Mobile Documents/com~apple~CloudDocs/PR iCloud/Work/Personal/Programming/Github Repos/hybrid-rl-index-trading/data/minutely_price_downloads/2025-05-19 23_13_44 DAX 4. close price data 2016-07-01 to 2025-04-30.csv


  4%|▍         | 1/24 [00:00<00:12,  1.85it/s]

Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.


  8%|▊         | 2/24 [00:01<00:11,  1.91it/s]

Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.


 12%|█▎        | 3/24 [00:01<00:08,  2.40it/s]

Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.


100%|██████████| 24/24 [00:12<00:00,  1.91it/s]


,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2014-10-23 09:31:00,19.8527,19.8527,19.8527,19.8527,100.0
2014-10-23 10:06:00,19.8606,19.8606,19.8606,19.8606,4900.0
2014-10-23 10:48:00,19.9001,19.9001,19.9001,19.9001,1000.0
2014-10-23 13:07:00,19.9951,19.9951,19.9951,19.9951,100.0
2014-10-23 13:31:00,19.9932,19.9932,19.9932,19.9932,600.0
...,...,...,...,...,...
2025-04-30 17:13:00,41.9000,41.9000,41.9000,41.9000,5.0
2025-04-30 18:00:00,41.5582,41.5582,41.5582,41.5582,50.0
2025-04-30 18:09:00,41.5100,41.5100,41.5100,41.5100,3.0


## Interpolation

In [18]:
# date import parameters:
date_column = 'date'
price_column = '4. close'
lateset_file_path = filemgmt.most_recent_file(MINUTELY_PRICES, ".csv", "DAX")

# interpolation parameters:
sampling_rate_to_be_interpolated = '15min'
manual_operating_hours = (8, 22)
custom_start_hour = 16
custom_start_minute = 0

interpolated_prices = preprocessing.time_interpolation_new_sampling_rate(df=pd.read_csv(lateset_file_path),
                                                                         interpolation_column=price_column,
                                                                         datetime_column=date_column,
                                                                         new_sampling_rate=sampling_rate_to_be_interpolated,
                                                                         custom_start_hour=custom_start_hour,
                                                                         custom_start_minute=custom_start_minute,
                                                                         verbose=True,
                                                                         manual_operating_hours=manual_operating_hours,
                                                                         save_path=INTERPOLATED_PRICES,
                                                                         save_title_identifier='DAX Close')
interpolated_prices

Excluded every entry on Saturday or Sunday.
Excluded every entry before hour 8 and after hour 22.
New sampling rate (15min) is higher than current lowest time unit (1min).
Therefore some existing indices will be removed to match the new sampling rate, while all information will be kept through the interpolation procedure.


,close
date,
2014-10-23 16:00:00,19.974245
2014-10-23 16:15:00,19.973079
2014-10-23 16:30:00,19.971912
2014-10-23 16:45:00,19.970746
2014-10-23 17:00:00,19.969579
...,...
2025-04-30 18:30:00,41.510000
2025-04-30 18:45:00,41.510000
2025-04-30 19:00:00,41.510000


In [19]:
def print_time_index_entries_p_day(df: pd.DataFrame) -> None:
    df = df.copy()
    df['counter'] = 1
    statistics = df.counter.groupby(by=[df.index.year, df.index.day_of_year]).sum().describe()
    print('Average entries p. day:\t\t\t\t\t', statistics['mean'])
    print('Min. entries p. day:\t\t\t\t\t', statistics['min'])
    print('Lowest 25% entries p. day are below:\t', statistics['25%'])
    print('Lowest 50% entries p. day are below:\t', statistics['50%'])
    print('Lowest 75% entries p. day are below:\t', statistics['75%'])
    print('Max. entries p. day:\t\t\t\t\t', statistics['max'])


# interesting statistics to decide the sampling rate:    
print_time_index_entries_p_day(interpolated_prices)

Average entries p. day:					 55.985063752276865
Min. entries p. day:					 24.0
Lowest 25% entries p. day are below:	 56.0
Lowest 50% entries p. day are below:	 56.0
Lowest 75% entries p. day are below:	 56.0
Max. entries p. day:					 56.0
